In [119]:
    # Configuration - Change these values as needed then run all cells
    ITEM_ID = "ESSENCE_WITHER"  # Change this to your desired item ID
    START_TIME = [2025, 9, 10, 0]   # [YEAR, MONTH, DAY, HOUR]
    END_TIME = [2025, 10, 17, 0]    # [YEAR, MONTH, DAY, HOUR]
    #END_TIME = [2024, 9, 15, 16]    # [YEAR, MONTH, DAY, HOUR]
    STEP = 1  # Number of hours to increment between queries 
    CSV_FILE = "ESSENCE_WITHER_prices.csv"

In [95]:
import requests
import csv
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any
import calendar

class SkyblockPriceTracker:
    def __init__(self, item_id: str, start_time: List[int], end_time: List[int], step: int = 1):
        self.item_id = item_id
        self.start_time = start_time
        self.end_time = end_time
        self.step = step
        self.base_url = "https://sky.coflnet.com/api/bazaar/{}/snapshot?timestamp={}"
        self.data = []
        
    def format_timestamp(self, year: int, month: int, day: int, hour: int) -> str:
        return f"{year:04d}-{month:02d}-{day:02d}T{hour:02d}:00"
    
    def increment_time(self, year: int, month: int, day: int, hour: int) -> tuple:
        # Increment hour by step
        hour += self.step
        
        # Handle hour overflow
        while hour > 23:
            hour -= 24
            day += 1
            
            # Get days in current month
            days_in_month = calendar.monthrange(year, month)[1]
            
            # Handle day overflow
            while day > days_in_month:
                day -= days_in_month
                month += 1
                
                # Handle month overflow
                if month > 12:
                    month = 1
                    year += 1
                
                # Get days in new month
                days_in_month = calendar.monthrange(year, month)[1]
                    
        return year, month, day, hour
    
    def is_time_before_end(self, current_time: List[int]) -> bool:
        current_datetime = datetime(current_time[0], current_time[1], current_time[2], current_time[3])
        end_datetime = datetime(self.end_time[0], self.end_time[1], self.end_time[2], self.end_time[3])
        
        return current_datetime <= end_datetime
    
    def query_api(self, timestamp: str) -> Dict[str, Any]:
        url = self.base_url.format(self.item_id, timestamp)
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error querying API for timestamp {timestamp}: {e}")
            return None
        except Exception as e:
            print(f"Unexpected error for timestamp {timestamp}: {e}")
            return None
    
    def collect_data(self):
        current_time = self.start_time.copy()
        query_count = 0
        
        print(f"Starting data collection for {self.item_id}")
        print(f"Time range: {self.start_time} to {self.end_time}")
        print(f"Step: {self.step} hour(s)")
        
        while self.is_time_before_end(current_time):
            timestamp = self.format_timestamp(*current_time)
            print(f"Querying: {timestamp}")
            
            data = self.query_api(timestamp)
            query_count += 1
            
            # Format date for CSV
            date_str = f"{current_time[0]:04d}-{current_time[1]:02d}-{current_time[2]:02d} {current_time[3]:02d}:00:00"
            
            if data and 'sellPrice' in data:
                # Extract all available fields from the API response
                row_data = {
                    'Date': date_str,
                    'Buy Price': data.get('buyPrice', None),
                    'Buy Volume': data.get('buyVolume', None),
                    'Buy Moving Week': data.get('buyMovingWeek', None),
                    'Buy Orders Count': data.get('buyOrdersCount', None),
                    'Sell Price': data.get('sellPrice', None),
                    'Sell Volume': data.get('sellVolume', None),
                    'Sell Moving Week': data.get('sellMovingWeek', None),
                    'Sell Orders Count': data.get('sellOrdersCount', None)
                }
                
                self.data.append(row_data)
                
                print(f"  Found data - Buy: {data.get('buyPrice', 'N/A')}, Sell: {data.get('sellPrice', 'N/A')}")
            else:
                # No data found - add entry with current date in sell price slot for compatibility
                self.data.append({
                    'Date': date_str,
                    'Buy Price': None,
                    'Buy Volume': None,
                    'Buy Moving Week': None,
                    'Buy Orders Count': None,
                    'Sell Price': str(current_time),  # Use current date list as placeholder
                    'Sell Volume': None,
                    'Sell Moving Week': None,
                    'Sell Orders Count': None
                })
                
                print(f"  No data found for {timestamp} - added date {current_time} as placeholder")
            
            # Increment time by step hours
            current_time = list(self.increment_time(*current_time))
            
            # Small delay to be respectful to the API
            time.sleep(0.1)
            
            # After 95 consecutive queries, wait 10 seconds
            if query_count % 95 == 0:
                print(f"  Completed {query_count} queries. Waiting 10 seconds before continuing...")
                time.sleep(10)
        
        print(f"Data collection complete. Found {len(self.data)} data points.")
    
    def save_to_csv(self, filename: str = None):
        if not filename:
            filename = f"{self.item_id}_prices.csv"
        
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = [
                'Date', 
                'Buy Price', 
                'Buy Volume', 
                'Buy Moving Week', 
                'Buy Orders Count',
                'Sell Price', 
                'Sell Volume', 
                'Sell Moving Week', 
                'Sell Orders Count'
            ]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            writer.writerows(self.data)
        
        CSV_FILE = filename
        print(f"Data saved to {filename}")


In [96]:
import ast
import pandas as pd

class CSVDataUpdater:
    def __init__(self, csv_filename: str, item_id: str):
        self.csv_filename = csv_filename
        self.item_id = item_id
        self.base_url = "https://sky.coflnet.com/api/bazaar/{}/snapshot?timestamp={}"
        
    def is_date_list(self, value):
        try:
            # Try to parse as a list
            parsed = ast.literal_eval(str(value))
            if isinstance(parsed, list) and len(parsed) == 4:
                # Check if all elements are integers (year, month, day, hour)
                if all(isinstance(x, int) for x in parsed):
                    return True
        except (ValueError, SyntaxError):
            pass
        return False
    
    def format_timestamp_from_list(self, date_list):
        year, month, day, hour = date_list
        return f"{year:04d}-{month:02d}-{day:02d}T{hour:02d}:00"
    
    def query_api_for_data(self, timestamp: str):
        url = self.base_url.format(self.item_id, timestamp)
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            
            if data and 'sellPrice' in data:
                # Return all available fields from the API response
                return {
                    'Buy Price': data.get('buyPrice', None),
                    'Buy Volume': data.get('buyVolume', None),
                    'Buy Moving Week': data.get('buyMovingWeek', None),
                    'Buy Orders Count': data.get('buyOrdersCount', None),
                    'Sell Price': data.get('sellPrice', None),
                    'Sell Volume': data.get('sellVolume', None),
                    'Sell Moving Week': data.get('sellMovingWeek', None),
                    'Sell Orders Count': data.get('sellOrdersCount', None)
                }
        except requests.exceptions.RequestException as e:
            if "Expecting value: line 1 column 1 (char 0)" in str(e):
                print(f"Empty response for timestamp {timestamp} - will remove row")
                return "REMOVE_ROW"  # Special marker to indicate row should be removed
            else:
                print(f"Error querying API for timestamp {timestamp}: {e}")
        except ValueError as e:
            if "Expecting value: line 1 column 1 (char 0)" in str(e):
                print(f"Empty response for timestamp {timestamp} - will remove row")
                return "REMOVE_ROW"  # Special marker to indicate row should be removed
            else:
                print(f"JSON decode error for timestamp {timestamp}: {e}")
        except Exception as e:
            if "Expecting value: line 1 column 1 (char 0)" in str(e):
                print(f"Empty response for timestamp {timestamp} - will remove row")
                return "REMOVE_ROW"  # Special marker to indicate row should be removed
            else:
                print(f"Unexpected error for timestamp {timestamp}: {e}")
        
        return None
    
    def update_csv_data(self):
        print(f"Reading CSV file: {self.csv_filename}")
        
        # Read the CSV file
        df = pd.read_csv(self.csv_filename)
        
        print(f"Found {len(df)} rows in CSV")
        
        # First, remove rows with no value in the buy price cell
        initial_count = len(df)
        
        # Remove rows where 'Buy Price' is NaN, None, empty string, or whitespace only
        if 'Buy Price' in df.columns:
            df = df.dropna(subset=['Buy Price'])  # Remove NaN values
            df = df[df['Buy Price'].astype(str).str.strip() != '']  # Remove empty strings and whitespace-only strings
        
        empty_removed_count = initial_count - len(df)
        if empty_removed_count > 0:
            print(f"Removed {empty_removed_count} rows with no value in buy price cell")
        
        # Find rows with date lists in buy price column
        date_list_indices = []
        for idx, row in df.iterrows():
            if 'Buy Price' in df.columns and self.is_date_list(row['Buy Price']):
                date_list_indices.append(idx)
        
        print(f"Found {len(date_list_indices)} entries with date lists to update")
        
        if len(date_list_indices) == 0:
            print("No date lists found in sell price column. Nothing to update.")
            # Still save the CSV in case we removed empty rows
            if empty_removed_count > 0:
                df.to_csv(self.csv_filename, index=False)
                print(f"Updated CSV saved to: {self.csv_filename}")
            return
        
        # Process each date list entry
        updated_count = 0
        removed_count = 0
        
        # Process rows in reverse order to avoid index shifting issues
        for idx in reversed(date_list_indices):
            date_list_str = df.loc[idx, 'Buy Price']
            date_list = ast.literal_eval(date_list_str)
            
            # Format timestamp for API
            timestamp = self.format_timestamp_from_list(date_list)
            print(f"Updating row {idx}: {timestamp}")
            
            # Query API for all data
            api_data = self.query_api_for_data(timestamp)
            
            if api_data == "REMOVE_ROW":
                # Remove row immediately
                df = df.drop(idx)
                removed_count += 1
                print(f"  Removed row {idx} due to empty API response")
            elif api_data is not None:
                # Update all columns with the new data
                for column, value in api_data.items():
                    df.loc[idx, column] = value
                updated_count += 1
                print(f"  Updated with data - Buy: {api_data.get('Buy Price', 'N/A')}, Sell: {api_data.get('Sell Price', 'N/A')}")
            else:
                print(f"  No data found for {timestamp}")
            
            # Small delay to be respectful to the API
            time.sleep(0.1)
        
        # Save the updated CSV
        df.to_csv(self.csv_filename, index=False)
        print(f"\nUpdated {updated_count} entries out of {len(date_list_indices)} date lists")
        print(f"Removed {removed_count} rows with empty API responses")
        print(f"Removed {empty_removed_count} rows with no value in buy price cell")
        print(f"Updated CSV saved to: {self.csv_filename}")

def update_csv_prices(csv_filename: str, item_id: str):
    updater = CSVDataUpdater(csv_filename, item_id)
    updater.update_csv_data()

# Example usage:
# update_csv_prices("GOLDEN_FRAGMENT_prices.csv", "GOLDEN_FRAGMENT")


In [97]:
import pandas as pd
from datetime import datetime
import requests

class MayorDataProcessor:
    def __init__(self, csv_filename: str, start_time: list, end_time: list):
        self.csv_filename = csv_filename
        self.start_time = start_time
        self.end_time = end_time
        self.mayor_url = "https://sky.coflnet.com/api/mayor?from={}&to={}"
        
    def format_timestamp_for_mayor_api(self, time_list):
        year, month, day, hour = time_list
        return f"{year:04d}-{month:02d}-{day:02d}T{hour:02d}:00"
    
    def fetch_mayor_data(self):
        from_timestamp = self.format_timestamp_for_mayor_api(self.start_time)
        to_timestamp = self.format_timestamp_for_mayor_api(self.end_time)
        
        url = self.mayor_url.format(from_timestamp, to_timestamp)
        print(f"Fetching mayor data from: {url}")
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            print(f"Successfully fetched {len(data)} mayor elections")
            return data
        except requests.exceptions.RequestException as e:
            print(f"Error fetching mayor data: {e}")
            return None
        except Exception as e:
            print(f"Unexpected error fetching mayor data: {e}")
            return None
    
    def parse_mayor_date(self, date_str):
        try:
            # Handle different date formats from the API
            if "+00:00" in date_str:
                # Format: "06/30/2024 11:15:00 +00:00"
                date_str = date_str.replace(" +00:00", "")
                return datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
            else:
                # Format: "07/05/2024 15:15:00"
                return datetime.strptime(date_str, "%m/%d/%Y %H:%M:%S")
        except Exception as e:
            print(f"Error parsing date '{date_str}': {e}")
            return None
    
    def get_mayor_for_timestamp(self, timestamp_str, mayor_elections):
        try:
            # Parse the CSV timestamp
            csv_date = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
            
            # Check each mayor election period
            for election in mayor_elections:
                start_date = self.parse_mayor_date(election['start'])
                end_date = self.parse_mayor_date(election['end'])
                
                if start_date and end_date:
                    # Check if CSV date falls within this mayor's term
                    if start_date <= csv_date <= end_date:
                        return election['winner']['name']
            
            return None
        except Exception as e:
            print(f"Error processing timestamp '{timestamp_str}': {e}")
            return None
    
    def add_mayor_column(self):
        print(f"Reading CSV file: {self.csv_filename}")
        
        # Read the CSV file
        df = pd.read_csv(self.csv_filename)
        print(f"Found {len(df)} rows in CSV")
        
        # Fetch mayor data
        mayor_elections = self.fetch_mayor_data()
        if not mayor_elections:
            print("Failed to fetch mayor data. Cannot proceed.")
            return
        
        # Add mayor column
        df['Mayor'] = None
        
        # Process each row
        for idx, row in df.iterrows():
            timestamp_str = row['Date']
            mayor_name = self.get_mayor_for_timestamp(timestamp_str, mayor_elections)
            df.loc[idx, 'Mayor'] = mayor_name
            
            if idx % 100 == 0:  # Progress indicator
                print(f"Processed {idx} rows...")
        
        # Save the updated CSV
        df.to_csv(self.csv_filename, index=False)
        print(f"\nSuccessfully added Mayor column to {self.csv_filename}")
        print(f"Mayor data added for {len(df)} rows")

def add_mayor_data_to_csv(csv_filename: str, start_time: list, end_time: list):
    processor = MayorDataProcessor(csv_filename, start_time, end_time)
    processor.add_mayor_column()

# Example usage:
# add_mayor_data_to_csv("GOLDEN_FRAGMENT_prices.csv", [2024, 7, 2, 16], [2025, 10, 16, 2])


In [98]:
import pandas as pd
import numpy as np

class MayorColumnConverter:
    def __init__(self, csv_filename: str):
        self.csv_filename = csv_filename
        
    def get_unique_mayors(self, df):
        # Get unique mayors, excluding None/NaN values
        unique_mayors = df['Mayor'].dropna().unique()
        # Sort for consistent column ordering
        return sorted(unique_mayors)
    
    def convert_mayor_to_binary_columns(self):
        print(f"Reading CSV file: {self.csv_filename}")
        
        # Read the CSV file
        df = pd.read_csv(self.csv_filename)
        print(f"Found {len(df)} rows in CSV")
        
        # Check if Mayor column exists
        if 'Mayor' not in df.columns:
            print("Error: 'Mayor' column not found in CSV file.")
            print("Available columns:", list(df.columns))
            return
        
        # Get unique mayors
        unique_mayors = self.get_unique_mayors(df)
        print(f"Found {len(unique_mayors)} unique mayors: {unique_mayors}")
        
        if len(unique_mayors) == 0:
            print("No mayors found in the Mayor column.")
            return
        
        # Create binary columns for each mayor
        for mayor in unique_mayors:
            # Create column name (replace spaces with underscores for better CSV compatibility)
            column_name = f"Mayor_{mayor.replace(' ', '_')}"
            # Set to 1 if this mayor was active, 0 otherwise
            df[column_name] = (df['Mayor'] == mayor).astype(int)
            print(f"Created column: {column_name}")
        
        # Remove the original Mayor column
        df = df.drop('Mayor', axis=1)
        print("Removed original 'Mayor' column")
        
        # Save the updated CSV
        df.to_csv(self.csv_filename, index=False)
        print(f"\nSuccessfully converted Mayor column to binary columns")
        print(f"New columns created: {[f'Mayor_{mayor.replace(' ', '_')}' for mayor in unique_mayors]}")
        print(f"Updated CSV saved to: {self.csv_filename}")
        
        # Show sample of the new data
        print(f"\nSample of new data:")
        print(df.head())
        
        return df

def convert_mayor_columns(csv_filename: str):
    converter = MayorColumnConverter(csv_filename)
    return converter.convert_mayor_to_binary_columns()

# Example usage:
# convert_mayor_columns("GOLDEN_FRAGMENT_prices.csv")


In [99]:
# Create the CSV file
tracker = SkyblockPriceTracker(ITEM_ID, START_TIME, END_TIME, STEP)
tracker.collect_data()
tracker.save_to_csv()
print("\nScript completed successfully!")

Starting data collection for ESSENCE_DIAMOND
Time range: [2025, 9, 10, 0] to [2025, 10, 17, 0]
Step: 1 hour(s)
Querying: 2025-09-10T00:00
  Found data - Buy: 327.9, Sell: 281.6
Querying: 2025-09-10T01:00
  Found data - Buy: 328.0, Sell: 281.6
Querying: 2025-09-10T02:00
  Found data - Buy: 328.1, Sell: 281.6
Querying: 2025-09-10T03:00
  Found data - Buy: 328.0, Sell: 281.4
Querying: 2025-09-10T04:00
  Found data - Buy: 328.0, Sell: 281.4
Querying: 2025-09-10T05:00
  Found data - Buy: 327.7, Sell: 281.5
Querying: 2025-09-10T06:00
  Found data - Buy: 328.0, Sell: 281.4
Querying: 2025-09-10T07:00
  Found data - Buy: 327.8, Sell: 281.3
Querying: 2025-09-10T08:00
  Found data - Buy: 322.2, Sell: 281.4
Querying: 2025-09-10T09:00
  Found data - Buy: 321.5, Sell: 281.5
Querying: 2025-09-10T10:00
  Found data - Buy: 321.3, Sell: 281.5
Querying: 2025-09-10T11:00
  Found data - Buy: 321.5, Sell: 281.5
Querying: 2025-09-10T12:00
  Found data - Buy: 321.4, Sell: 281.7
Querying: 2025-09-10T13:00
  Fo

In [113]:
# Example: Update your CSV file with missing prices
update_csv_prices(CSV_FILE, ITEM_ID)

Reading CSV file: ESSENCE_CRIMSON_prices.csv
Found 888 rows in CSV
Found 0 entries with date lists to update
No date lists found in sell price column. Nothing to update.


In [101]:
# Add mayor data to your CSV file
add_mayor_data_to_csv(CSV_FILE, START_TIME, END_TIME)


Reading CSV file: ESSENCE_DIAMOND_prices.csv
Found 734 rows in CSV
Fetching mayor data from: https://sky.coflnet.com/api/mayor?from=2025-09-10T00:00&to=2025-10-17T00:00
Successfully fetched 8 mayor elections
Processed 0 rows...
Processed 100 rows...
Processed 200 rows...
Processed 300 rows...
Processed 400 rows...
Processed 500 rows...
Processed 600 rows...
Processed 700 rows...

Successfully added Mayor column to ESSENCE_DIAMOND_prices.csv
Mayor data added for 734 rows


In [102]:
# Convert mayor column to binary columns
convert_mayor_columns(CSV_FILE)


Reading CSV file: ESSENCE_DIAMOND_prices.csv
Found 734 rows in CSV
Found 7 unique mayors: ['Aatrox', 'Cole', 'Diana', 'Diaz', 'Jerry', 'Marina', 'Paul']
Created column: Mayor_Aatrox
Created column: Mayor_Cole
Created column: Mayor_Diana
Created column: Mayor_Diaz
Created column: Mayor_Jerry
Created column: Mayor_Marina
Created column: Mayor_Paul
Removed original 'Mayor' column

Successfully converted Mayor column to binary columns
New columns created: ['Mayor_Aatrox', 'Mayor_Cole', 'Mayor_Diana', 'Mayor_Diaz', 'Mayor_Jerry', 'Mayor_Marina', 'Mayor_Paul']
Updated CSV saved to: ESSENCE_DIAMOND_prices.csv

Sample of new data:
                  Date  Buy Price  Buy Volume  Buy Moving Week  \
0  2025-09-10 00:00:00      327.9   1101754.0       10174776.0   
1  2025-09-10 01:00:00      328.0   1081840.0       10152883.0   
2  2025-09-10 02:00:00      328.1   1059624.0       10094130.0   
3  2025-09-10 03:00:00      328.0   1132352.0       10057864.0   
4  2025-09-10 04:00:00      328.0   109

,Date,Buy Price,Buy Volume,Buy Moving Week,Buy Orders Count,Sell Price,Sell Volume,Sell Moving Week,Sell Orders Count,Mayor_Aatrox,Mayor_Cole,Mayor_Diana,Mayor_Diaz,Mayor_Jerry,Mayor_Marina,Mayor_Paul
0,2025-09-10 00:00:00,327.9,1101754.0,10174776.0,20.0,281.6,583015.0,8871768.0,14.0,1,0,0,0,0,0,0
1,2025-09-10 01:00:00,328.0,1081840.0,10152883.0,20.0,281.6,545797.0,8858689.0,13.0,1,0,0,0,0,0,0
2,2025-09-10 02:00:00,328.1,1059624.0,10094130.0,19.0,281.6,510704.0,8864187.0,13.0,1,0,0,0,0,0,0
3,2025-09-10 03:00:00,328.0,1132352.0,10057864.0,21.0,281.4,464152.0,8881925.0,12.0,1,0,0,0,0,0,0
4,2025-09-10 04:00:00,328.0,1098055.0,10079412.0,20.0,281.4,450366.0,8891293.0,11.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,2025-10-16 20:00:00,444.2,16122318.0,9612836.0,320.0,295.1,675464.0,10140004.0,16.0,0,0,0,0,1,0,0
730,2025-10-16 21:00:00,442.8,16527389.0,9610035.0,334.0,294.5,602415.0,10158710.0,14.0,0,0,0,0,1,0,0
731,2025-10-16 22:00:00,426.9,16820806.0,9571485.0,346.0,294.5,583549.0,10184063.0,14.0,0,0,0,0,1,0,0
732,2025-10-16 23:00:00,426.9,16920884.0,9533219.0,346.0,294.5,514148.0,10202625.0,13.0,0,0,0,0,1,0,0


In [103]:
import pandas as pd
import numpy as np

class PriceChangeCalculator:
    def __init__(self, csv_filename: str):
        self.csv_filename = csv_filename
        
    def calculate_price_change_percentage(self):
        print(f"Reading CSV file: {self.csv_filename}")
        
        # Read the CSV file
        df = pd.read_csv(self.csv_filename)
        print(f"Found {len(df)} rows in CSV")
        
        # Check if Sell Price column exists
        if 'Sell Price' not in df.columns:
            print("Error: 'Sell Price' column not found in CSV file.")
            print("Available columns:", list(df.columns))
            return
        
        # Convert Sell Price to numeric, handling any non-numeric values
        df['Sell Price'] = pd.to_numeric(df['Sell Price'], errors='coerce')
        
        # Calculate percentage change
        # pct_change() calculates (current - previous) / previous * 100
        df['Price_Change_Percent'] = df['Sell Price'].pct_change() * 100
        
        # Round to 2 decimal places for readability
        df['Price_Change_Percent'] = df['Price_Change_Percent'].round(2)
        
        # The first row will have NaN since there's no previous price to compare to
        # You can either keep it as NaN or set it to 0
        df['Price_Change_Percent'] = df['Price_Change_Percent'].fillna(0)
        
        # Save the updated CSV
        df.to_csv(self.csv_filename, index=False)
        print(f"\nSuccessfully added Price_Change_Percent column")
        print(f"Updated CSV saved to: {self.csv_filename}")
        
        # Show statistics about the price changes
        price_changes = df['Price_Change_Percent'].dropna()
        if len(price_changes) > 0:
            print(f"\nPrice Change Statistics:")
            print(f"  Average change: {price_changes.mean():.2f}%")
            print(f"  Median change: {price_changes.median():.2f}%")
            print(f"  Max increase: {price_changes.max():.2f}%")
            print(f"  Max decrease: {price_changes.min():.2f}%")
            print(f"  Standard deviation: {price_changes.std():.2f}%")
        
        # Show sample of the new data
        print(f"\nSample of new data:")
        print(df.head(10))
        
        return df

def add_price_change_column(csv_filename: str):
    calculator = PriceChangeCalculator(csv_filename)
    return calculator.calculate_price_change_percentage()

# Example usage:
# add_price_change_column("GOLDEN_FRAGMENT_prices.csv")


In [104]:
# Add price change percentage column
add_price_change_column(CSV_FILE)


Reading CSV file: ESSENCE_DIAMOND_prices.csv
Found 734 rows in CSV

Successfully added Price_Change_Percent column
Updated CSV saved to: ESSENCE_DIAMOND_prices.csv

Price Change Statistics:
  Average change: 1.55%
  Median change: 0.04%
  Max increase: 1173.84%
  Max decrease: -93.94%
  Standard deviation: 43.55%

Sample of new data:
                  Date  Buy Price  Buy Volume  Buy Moving Week  \
0  2025-09-10 00:00:00      327.9   1101754.0       10174776.0   
1  2025-09-10 01:00:00      328.0   1081840.0       10152883.0   
2  2025-09-10 02:00:00      328.1   1059624.0       10094130.0   
3  2025-09-10 03:00:00      328.0   1132352.0       10057864.0   
4  2025-09-10 04:00:00      328.0   1098055.0       10079412.0   
5  2025-09-10 05:00:00      327.7   1206199.0       10059918.0   
6  2025-09-10 06:00:00      328.0   1037875.0       10183040.0   
7  2025-09-10 07:00:00      327.8   1032508.0       10179484.0   
8  2025-09-10 08:00:00      322.2   1166126.0       10095776.0   
9  2

,Date,Buy Price,Buy Volume,Buy Moving Week,Buy Orders Count,Sell Price,Sell Volume,Sell Moving Week,Sell Orders Count,Mayor_Aatrox,Mayor_Cole,Mayor_Diana,Mayor_Diaz,Mayor_Jerry,Mayor_Marina,Mayor_Paul,Price_Change_Percent
0,2025-09-10 00:00:00,327.9,1101754.0,10174776.0,20.0,281.6,583015.0,8871768.0,14.0,1,0,0,0,0,0,0,0.00
1,2025-09-10 01:00:00,328.0,1081840.0,10152883.0,20.0,281.6,545797.0,8858689.0,13.0,1,0,0,0,0,0,0,0.00
2,2025-09-10 02:00:00,328.1,1059624.0,10094130.0,19.0,281.6,510704.0,8864187.0,13.0,1,0,0,0,0,0,0,0.00
3,2025-09-10 03:00:00,328.0,1132352.0,10057864.0,21.0,281.4,464152.0,8881925.0,12.0,1,0,0,0,0,0,0,-0.07
4,2025-09-10 04:00:00,328.0,1098055.0,10079412.0,20.0,281.4,450366.0,8891293.0,11.0,1,0,0,0,0,0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,2025-10-16 20:00:00,444.2,16122318.0,9612836.0,320.0,295.1,675464.0,10140004.0,16.0,0,0,0,0,1,0,0,0.03
730,2025-10-16 21:00:00,442.8,16527389.0,9610035.0,334.0,294.5,602415.0,10158710.0,14.0,0,0,0,0,1,0,0,-0.20
731,2025-10-16 22:00:00,426.9,16820806.0,9571485.0,346.0,294.5,583549.0,10184063.0,14.0,0,0,0,0,1,0,0,0.00
732,2025-10-16 23:00:00,426.9,16920884.0,9533219.0,346.0,294.5,514148.0,10202625.0,13.0,0,0,0,0,1,0,0,0.00


In [ ]:
import pandas as pd
import numpy as np

def calculate_price_statistics(csv_filename: str):
    print(f"Reading CSV file: {csv_filename}")
    
    # Read the CSV file
    df = pd.read_csv(csv_filename)
    print(f"Found {len(df)} rows in CSV")
    
    # Check if Sell Price column exists
    if 'Sell Price' not in df.columns:
        print("Error: 'Sell Price' column not found in CSV file.")
        print("Available columns:", list(df.columns))
        return None
    
    # Convert Sell Price to numeric, handling any non-numeric values
    df['Sell Price'] = pd.to_numeric(df['Sell Price'], errors='coerce')
    
    # Remove any rows with NaN sell prices
    valid_prices = df['Sell Price'].dropna()
    
    if len(valid_prices) == 0:
        print("No valid sell prices found in the data.")
        return None
    
    # Calculate statistics
    average_sell_price = valid_prices.mean()
    price_90_percent_below = average_sell_price * 0.9
    price_110_percent_above = average_sell_price * 1.1
    
    # Additional statistics
    min_price = valid_prices.min()
    max_price = valid_prices.max()
    median_price = valid_prices.median()
    std_deviation = valid_prices.std()
    
    # Display results
    print(f"\n=== PRICE STATISTICS FOR {csv_filename} ===")
    print(f"Total data points: {len(valid_prices)}")
    print(f"Average Sell Price: {average_sell_price:.2f}")
    print(f"90% of Average Price: {price_90_percent_below:.2f}")
    print(f"110% of Average Price: {price_110_percent_above:.2f}")
    print(f"\nAdditional Statistics:")
    print(f"Minimum Price: {min_price:.2f}")
    print(f"Maximum Price: {max_price:.2f}")
    print(f"Median Price: {median_price:.2f}")
    print(f"Standard Deviation: {std_deviation:.2f}")
    
    # Count how many prices fall in each range
    prices_below_90 = len(valid_prices[valid_prices <= price_90_percent_below])
    prices_above_110 = len(valid_prices[valid_prices >= price_110_percent_above])
    prices_between = len(valid_prices[(valid_prices > price_90_percent_below) & (valid_prices < price_110_percent_above)])
    
    print(f"\nPrice Distribution:")
    print(f"Prices ≤ 90% of average ({price_90_percent_below:.2f}): {prices_below_90} ({prices_below_90/len(valid_prices)*100:.1f}%)")
    print(f"Prices between 90%-110% of average: {prices_between} ({prices_between/len(valid_prices)*100:.1f}%)")
    print(f"Prices ≥ 110% of average ({price_110_percent_above:.2f}): {prices_above_110} ({prices_above_110/len(valid_prices)*100:.1f}%)")
    
    return {
        'average_price': average_sell_price,
        'price_90_percent_below': price_90_percent_below,
        'price_110_percent_above': price_110_percent_above,
        'min_price': min_price,
        'max_price': max_price,
        'median_price': median_price,
        'std_deviation': std_deviation,
        'total_data_points': len(valid_prices)
    }

# Calculate price statistics for the current CSV file
price_stats = calculate_price_statistics(CSV_FILE)


Reading CSV file: ESSENCE_WITHER_prices.csv
Found 879 rows in CSV

=== PRICE STATISTICS FOR ESSENCE_WITHER_prices.csv ===
Total data points: 876
Average Sell Price: 2527.06
90% of Average Price: 2274.36
110% of Average Price: 2779.77

Additional Statistics:
Minimum Price: 2001.60
Maximum Price: 2919.20
Median Price: 2475.70
Standard Deviation: 190.07

Price Distribution:
Prices ≤ 90% of average (2274.36): 49 (5.6%)
Prices between 90%-110% of average: 695 (79.3%)
Prices ≥ 110% of average (2779.77): 132 (15.1%)
